# RLDT

## Step 1: Import the necessary libraries:

In [781]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

## Step 2: Define the environment:

### Step 2.1: Devices

#### *Gloabl variables*

In [782]:
num_IOT_devices = 10

num_IOT_devices = 10

num_IOT_devices = 10

voltages_frequencies_IOT = [
    (1e6 * 50 , 1.8),
    (2e6 * 50 , 2.3),
    (4e6 * 50 , 2.7),
    (8e6 * 50 , 4.0),
    (16e6 * 50 , 5.0),
    
]
num_MEC_devices = 5

voltages_frequencies_MEC = [
    (6e8 /  1.15, 0.8),
    (7.5e8/ 1.15, 0.825),
    (10e8 / 1.15,  1.0),
    (15e8 / 1.15,  1.2),

]

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5

num_dag_generations = 10000

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5

num_dag_generations = 10000

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5

num_dag_generations = 10000

#### *IOT*

In [783]:
devices_data_IOT = []
for i in range(num_IOT_devices):
    cpu_cores = np.random.choice([4, 6, 8])
    device_info = {
        "id": i,
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [np.random.choice([0, 1]) for _ in range(cpu_cores)],
        "voltages_frequencies": [
            [
                voltages_frequencies_IOT[i]
                for i in np.random.choice(5, size=2, replace=False)
            ]
            for core in range(cpu_cores)
        ],
        "ISL": np.random.randint(10, 21),
        "capacitance": [np.random.uniform(2, 3) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [
            np.random.choice([700, 800, 900]) * 1e-6 for _ in range(cpu_cores)
        ],
        "batteryLevel": np.random.randint(36, 41) * 1e9,
        "errorRate": np.random.randint(1, 6) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.25, 0.75]),
    }
    devices_data_IOT.append(device_info)

IoTdevices = pd.DataFrame(devices_data_IOT)

IoTdevices.set_index("id", inplace=True)
IoTdevices["name"] = "iot"


#### *MEC*

In [784]:
devices_data_MEC = []
for i in range(num_MEC_devices):
    cpu_cores = np.random.choice([16,32,64])
    device_info = {
        "id": i,
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [np.random.choice([0, 1]) for _ in range(cpu_cores)],
        "voltages_frequencies": [[
            voltages_frequencies_MEC[i]
            for i in np.random.choice(4, size=2, replace=False)
        ]for core in range(cpu_cores)],
        "capacitance": [np.random.uniform(1.5, 2) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [np.random.choice([9, 9, 10]) * 1e-5 for _ in range(cpu_cores)],
        "errorRate": np.random.randint(5, 11) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.75, 0.25]),
        "batteryLevel": 100,
        "ISL": 0,
    }
    devices_data_MEC.append(device_info)

MECDevices = pd.DataFrame(devices_data_MEC)

MECDevices.set_index("id", inplace=True)
MECDevices["name"] = "mec"
# MECDevices

#### *CLOUD*

In [785]:
cloud_configurations = [(1,13.85), (2,3.9e9,24.28)]
device_info = [
    {
        "id": 0,
        "number_of_cpu_cores": 1,
        "occupied_cores": [0],
        "voltages_frequencies": [[2.8e9, 3.9e9]],
        "capacitance": (13.85, 24.28),
        "powerIdle": 0,
        "ISL": 0,
        "batteryLevel": 100,
        "errorRate": 0.1,
        "accetableTasks": [1, 2, 3, 4],
        "handleSafeTask": 0,
    }
]
cloud = pd.DataFrame(device_info)
cloud=cloud.set_index("id")
cloud["name"] = "cloud"

####        ALL THE DEVICES

In [786]:
# devices = pd.concat([IoTdevices,MECDevices,cloud],ignore_index=True)
devices = pd.concat([IoTdevices],ignore_index=True)
# devices = pd.concat([MECDevices],ignore_index=True)
# devices = pd.concat([cloud],ignore_index=True)
# devices

### Step 2.2: Application

#### *helper function : generate_random_dag*

In [787]:
def generate_random_dag(num_nodes):
    dag = nx.DiGraph()

    nodes = [f"t{i+1}" for i in range(num_nodes)]
    dag.add_nodes_from(nodes)

    available_parents = {node: list(nodes[:i]) for i, node in enumerate(nodes)}

    for i in range(2, num_nodes + 1):
       
        num_parents = min(
            random.randint(1, min(i, max_num_parents_dag)), len(available_parents[f"t{i}"])
        )

        # select parents
        parent_nodes = random.sample(available_parents[f"t{i}"], num_parents)
        # add parents
        dag.add_edges_from((parent_node, f"t{i}") for parent_node in parent_nodes)

        # update available parents
        available_parents[f"t{i}"] = list(nodes[:i])

    return dag

#### *Generate task DAGs*

In [788]:
tasks_data = []

start_node_number = 1
for run in range(num_dag_generations):

    num_nodes = random.randint(min_num_nodes_dag, max_num_nodes_dag)

    random_dag = generate_random_dag(num_nodes)

    mapping = {
        f"t{i}": f"t{i + start_node_number - 1}" for i in range(1, num_nodes + 1)
    }

    random_dag = nx.relabel_nodes(random_dag, mapping)
    for node in random_dag.nodes:
        parents = list(random_dag.predecessors(node))
        task_info = {
            "id": node,
            "job": run,
            "dependency": parents,
            "mobility": np.random.randint(1, 10),
            "kind": np.random.choice(task_kinds),
            "safe": np.random.choice([0, 1], p=[0.95, 0.05]),
            "computationalLoad": int(np.random.uniform(1, 100) * 1e4),
            "dataEntrySize": (np.random.randint(10, 100) // 10)* 1e9,
            "returnDataSize": (np.random.randint(10, 100) // 10)* 1e9,
            "status": "READY",
        }
        tasks_data.append(task_info)
    start_node_number += num_nodes

np.random.shuffle(tasks_data)
tasks = pd.DataFrame(tasks_data)

tasks.set_index("id", inplace=True)
# tasks

## Step 4 : DDT

### Step 4.1:  Initializing The tree

In [789]:
class DDT(nn.Module):
    def __init__(self, num_input, num_output, depth, max_depth):
        super(DDT, self).__init__()
        self.depth = depth
        self.max_depth = max_depth
        if depth != max_depth:
            # self.weights = nn.Parameter(torch.zeros(num_input))
            self.weights = nn.Parameter(torch.empty(num_input).normal_(mean=0, std=0.1))
            self.bias = nn.Parameter(torch.zeros(1))
        if depth == max_depth:
            self.prob_dist = nn.Parameter(torch.zeros(num_output))

        if depth < max_depth:
            self.left = DDT(num_input, num_output, depth + 1, max_depth)
            self.right = DDT(num_input, num_output, depth + 1, max_depth)

    def forward(self, x):
        if self.depth == self.max_depth:
            return self.prob_dist.softmax(dim=0)
        val = torch.sigmoid(torch.matmul(x, self.weights.t()) + self.bias)
        a = np.random.uniform(0, 1)
        if a < 0.1:
            val = 1 - val
        if val>=0.5:

            return val * self.right(x)
        else:

            return (1 - val) * self.left(x)

In [790]:
# for taskIndex in proccessingQueue:
#     minCost = float("inf")
#     coreAndDevice = None
#     NOIWithTheSameCost = 0;
#     for option in cloud:
#         frequency=option[0]
#         mode=option[0]

#     #  n microsecond
#         timeDownMec = (tasks.loc[taskIndex, "returnDataSize"] / 1e3) + 5e3
#         timeUpMec = (tasks.loc[taskIndex, "dataEntrySize"] / 1e3) + 5e3
#         # 10^4 ; fiber optics transfer rate per micro second;1* 10^3 fiberoptic latency in microsecond
#         timeDownCC = (tasks.loc[taskIndex, "returnDataSize"] / 1e4) + 1e3
#         timeUpCC = (tasks.loc[taskIndex, "dataEntrySize"] / 1e4) + 1e3
#         totalTime = timeUpCC+ timeUpMec + time + timeDownMec+timeDownCC

#         powerUpMec = 4.38412 * timeUpMec
#         powerDownMec = 4.38412 * timeDownMec

#         energyUpMec = powerUpMec * timeUpMec
#         energyDownMec = powerDownMec * timeDownMec
#         energyUpCC = 3.65 * timeUpCC
#         energyDownCC = 3.65 * timeDownCC
#         totalEnergy = energyUpCC+ energyUpMec + energy + energyDownMec+energyDownCC
#         cost = 99.5 * totalEnergy + 0.5 * totalTime
#         if cost < minCost:
#             minCost = cost
#             coreAndDevice = (option, cost)
#         elif cost==minCost:
#             NOIWithTheSameCost += 1
        
            
#     print("Task :",taskIndex,"Scheduled to mode : " ,option[1])

## Step 5: RL

### title1

In [791]:
def getStatusOfDependcy(dependency):
    result = 0
    for dep in dependency:
        result += tasks_copy.loc[dep, "status"]
    return result


tasks_copy = tasks.copy()
tasks_copy["status"] = tasks_copy["status"].map(
    {"WAIT": 0, "READY": 1, "QUEUED": 2, "SCEDULED": 3}
)
tasks_copy["dependency"] = tasks_copy["dependency"].apply(getStatusOfDependcy)

taskList = tasks_copy.index.tolist()
devices_copy = devices.copy()

### !! transmission !!

In [792]:
def calc_execution_time(device, task, core,dvfs):
    # in micro-seconds
    if device['number_of_cpu_cores'] ==1:
        return task["computationalLoad"] / device["voltages_frequencies"][0][0] * 1e-6
    return task["computationalLoad"] / device["voltages_frequencies"][core][dvfs][0] * 1e-6


def calc_power_consumption(device,task,core,dvfs):
    # in W * micro-seconds
    if device["number_of_cpu_cores"] == 1:
       return device["capacitance"][0] * calc_execution_time(device, task, core, dvfs)
    return (
        device["capacitance"][core]
        * (device["voltages_frequencies"][core][dvfs][1] ** 2)
        * device["voltages_frequencies"][core][dvfs][0]
        * calc_execution_time(device, task, core, dvfs)
    )

def calc_transfer_time(device, task, time, energy):

    device_type = device["name"]

    timeDownMec = task["returnDataSize"] / 1e9
    timeDownMec *= 1e-3
    timeDownMec += 5e-3
    timeDownMec *= 1e-3
    timeUpMec = task["dataEntrySize"] / 1e9
    timeUpMec *= 1e-3
    timeUpMec += 5e-3
    timeUpMec *= 1e-3

    # (this.alpha * this.transferRate / Math.pow(10, 6) + this.beta);
    alpha = 52e-5
    beta = 3.86412

    powerMec = alpha * 1e9 / 1e6 + beta

    timeDownCC = task["returnDataSize"] / 1e9
    timeDownCC *= 1e-3
    timeDownCC += 1e-3
    timeDownCC *= 1e-3
    timeUpCC = task["dataEntrySize"] / 1e9
    timeUpCC *= 1e-3
    timeUpCC += 1e-3
    timeUpCC *= 1e-3

    energyUpMec = powerMec * timeUpMec
    energyDownMec = powerMec * timeDownMec
    energyUpCC = 3.65 * timeUpCC
    energyDownCC = 3.65 * timeDownCC

    if device_type == "cloud":
        totalTime = timeUpCC+ timeUpMec + time + timeDownMec+timeDownCC
        totalEnergy = energyUpCC+ energyUpMec + energy + energyDownMec+energyDownCC

    elif device_type == "mec":
        totalTime = timeUpMec + time + timeDownMec
        totalEnergy =  energyUpMec + energy + energyDownMec

    elif device_type == "iot":
        totalTime = time
        totalEnergy = energy

    return totalTime, totalEnergy

In [793]:
taskList = tasks_copy.index.tolist()

class Environment:
    def __init__(self):
        self.total_avg = 0
        self.totalFail = 0
        self.feature_size = 9
        self.num_actions = len(devices)
        self.max_depth = 3
        self.agent = DDT(self.feature_size, self.num_actions, depth=0, max_depth=self.max_depth)
        self.optimizer = optim.Adam(self.agent.parameters(), lr=0.005)

    def execute_action(self, state, action):
        ttt = taskList[0]
        taskList.pop(0)

        device = devices_copy.iloc[action]
        
        checkAvailableCoree = (sum(device["occupied_cores"]) != device["number_of_cpu_cores"])
        checkIfSutible = False
        if state['kind'] in device["accetableTasks"]:
            checkIfSutible = True
        
        
        if (checkIfSutible):  
            for coreIndex in range(len(device["occupied_cores"])):
                if device["occupied_cores"][coreIndex] ==0:
                    e = calc_power_consumption(device, state,coreIndex, 0)
                    t = calc_execution_time(device, state,coreIndex, 0)
                    total_t , total_e = calc_transfer_time(device, state, t, e)
                    reward = -1 * total_t + -1 * total_e
                    
                    t *= 1e11
                    total_t *= 1e11
                    added_time = total_t - t
                    # reward = -1 * t + -1 * e
                    # print("Reward: ",reward, )
                    # print(f'reward: {reward * 1}, Device: {device["voltages_frequencies"][coreIndex][0]},  task : {tasks.loc[ttt]} , t: {total_t} e: {total_e}')
                    # return (tasks_copy.loc[taskList[0]], reward * 1,total_t,total_e)
                    return (tasks_copy.loc[taskList[0]], reward * 1,t,e, added_time, t)
 
        self.totalFail += 1
        return (tasks_copy.loc[taskList[0]],1,0,0,0,0)

    def train(self, num_epoch, num_episodes):
        for i in range(num_epoch):
            # print("memoryview")
            total_loss = 0
            
            env.totalFail = 0
            totalTime = 0
            for j in range(num_episodes):
                state = tasks_copy.loc[taskList[0]]
                x = torch.tensor(np.array(state.values, dtype=np.float32)).unsqueeze(0)

                output = self.agent(x)
                action_probabilities = torch.softmax(output, dim=0)

                # Sample an action based on the output probabilities
                action_index = torch.multinomial(torch.softmax(output, dim=0), 1).item()
                next_state, reward, t, e, added_t,og_t = self.execute_action(state, action_index)
                
                # Calculate the loss as the negative log probability of the chosen action multiplied by the reward
                loss = (
                    output[action_index] * reward
                ) 
                # print("meow")
                # print(loss)
                # print(output,action_index,reward,loss)
                total_loss += loss
                totalTime += t
            self.optimizer.zero_grad()
            avg_loss = total_loss/num_episodes
            avg_time = totalTime / num_episodes
            env.total_avg += avg_time
            avg_loss.backward()

            self.optimizer.step()
            if i % 1 == 0:
                print(f"Epoch {i+1} // avg time: {avg_time} total fail: {env.totalFail} // Average Loss: {avg_loss}// ")
                env.totalFail = 0
            # for name, param in env.agent.named_parameters():
            # print(name, param.grad)

env = Environment()
tree = env.agent
env.train(101, 100)

print('///////////////////')
print(f'total avg: {env.total_avg / 100}')
for name, param in env.agent.named_parameters():
    if "prob_dist" or "bias" not in name:
        # print(name,param)
        pass

Epoch 1 // avg time: 253.61748624999993 total fail: 41 // Average Loss: 0.04099998623132706// 
Epoch 2 // avg time: 261.4856275 total fail: 27 // Average Loss: 0.026942111551761627// 
Epoch 3 // avg time: 313.55437375 total fail: 37 // Average Loss: 0.03677568957209587// 
Epoch 4 // avg time: 226.88309000000015 total fail: 39 // Average Loss: 0.03866355121135712// 
Epoch 5 // avg time: 298.89809875 total fail: 36 // Average Loss: 0.03571994602680206// 
Epoch 6 // avg time: 228.42775625 total fail: 36 // Average Loss: 0.035384513437747955// 
Epoch 7 // avg time: 315.50321 total fail: 33 // Average Loss: 0.0323721282184124// 
Epoch 8 // avg time: 173.60621749999999 total fail: 36 // Average Loss: 0.0352485291659832// 
Epoch 9 // avg time: 276.7795474999999 total fail: 30 // Average Loss: 0.029326390475034714// 
Epoch 10 // avg time: 218.5652375 total fail: 45 // Average Loss: 0.04377976059913635// 
Epoch 11 // avg time: 270.75733000000014 total fail: 35 // Average Loss: 0.033594585955142